In [130]:
# # criar o navegador
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
import os

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

caminho = os.getcwd()
arquivo = caminho + '/index.html'

navegador = webdriver.Chrome()
navegador.get(f'file://{arquivo}')

In [131]:
# importar a base de dados
import pandas as pd

tabela = pd.read_excel("Processos.xlsx")
tabela["Status"] = tabela["Status"].astype(str)
display(tabela)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,nan
1,João,Lawyer Alon,EB3792,Rio de Janeiro,nan
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,nan
3,Carol,Amanda,PC5197,São Paulo,nan


In [132]:
for indice in tabela.index:
  elemento = navegador.find_element(By.CLASS_NAME, "dropdown-menu")
  ActionChains(navegador).move_to_element(elemento).perform()

  cidade = tabela.loc[indice, "Cidade"]
  navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()

  abas = navegador.window_handles
  navegador.switch_to.window(abas[1])
  
  navegador.find_element(By.ID, 'nome').send_keys(tabela.loc[indice, 'Nome'])
  navegador.find_element(By.ID, 'advogado').send_keys(tabela.loc[indice, 'Advogado'])
  navegador.find_element(By.ID, 'numero').send_keys(tabela.loc[indice, 'Processo'])
  navegador.find_element(By.CLASS_NAME, 'registerbtn').click()

  alerta = Alert(navegador)
  alerta.accept()

  # Aguarde até que o alerta apareça
  alerta = WebDriverWait(navegador, 10).until(EC.alert_is_present())

  if "Processo encontrado com sucesso" in alerta.text:
    tabela.loc[indice, 'Status'] = 'Encontrado'
  else:
    tabela.loc[indice, 'Status'] = 'Não encontrado'

  alerta.accept()
  navegador.close()
  navegador.switch_to.window(abas[0])


In [133]:
navegador.quit()
display(tabela)

# tabela.to_excel("Processos Atualizado.xlsx")

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Encontrado
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Encontrado
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Encontrado
3,Carol,Amanda,PC5197,São Paulo,Não encontrado
